As a kind of test, I've been provided with some test cases to verify the functionality of different components of the perceptron network as they're being created. We'll instantiate the components in the required configuration for the tests, and then observe whether they perform as expected.

In [117]:
import itertools
from typing import Callable
from inspect import signature

import numpy as np
import pandas as pd

import ml

<h2>Perceptron</h2>

First off, we require tests the implemented perceptron unit. The provided scenarios to verify compliance with the expected behaviour of perceptron are several logical gates, consisting of either one or a network of perceptrons. The following scenarios are expected:

<ol>
<li>An INVERT gate</li>
<li>An AND gate</li>
<li>An OR gate</li>
<li>A NOR gate</li>
<li>A non-specified "more complex" decision system with at least 3 inputs.</li>
</ol>

We'll first establish a framework that allows us to easily construct truth tables for boolean operations. Then we'll compare these with the results from our perceptrons and perceptron-networks to verify their validity.


In [118]:
def binary_input_space(length: int) -> pd.DataFrame:
    """Compute all possible binary input combinations with a certain length.

    Args:
        length: length of the combinations.

    Returns:
        DataFrame with all possible combinations of 0s and 1s within specified length."""
    return pd.DataFrame(list(itertools.product([0, 1], repeat=length))).rename(columns=lambda n: f"Input {n}")

In [119]:
def apply_operation(input_space: pd.DataFrame,
                    operation: Callable) -> pd.Series:
    """Apply a boolean operation to an input space as created by binary_input_space.

    Args:
        input_space: inputs to apply operation on.
        operation: boolean operation to apply.

    Returns:
        Series object with indexes corresponding to original input_space, with result of all boolean operations."""
    return input_space.apply(lambda row: operation(*row), axis=1)

def apply_perceptron(input_space: pd.DataFrame,
                     p_s: ml.InOutPutNetworkI):
    """Apply a Perceptron, PerceptronLayer or PerceptronNetwork to an input space as created by binary_input_space.

    Args:
        input_space: inputs to apply operation on.
        p_s: Perceptron, PerceptronLayer or PerceptronNetwork to feed input into.

    Returns:
        Series object with indexes corresponding to original input_space, with result of all inputs applied to p_s.
        If p_s returns multiple outputs, they are stored in tuple."""
    result = input_space.apply(lambda row: p_s.feed_forward(row), axis=1)
    if result.dtype == np.ndarray and result.iloc[0].size > 1:  # If individual results are numpy arrays that contain more then 1 element, cast them to tuples to allow for comparisons.
        result = result.apply(tuple)
    return result

In [120]:
def verify_perceptron_operation_inputs(operation: Callable,
                                       p_s: ml.InOutPutNetworkI) -> None:
    """Verify that a boolean operation and a Perceptron, PerceptronLayer or PerceptronNetwork expect the same number of inputs.

    Args:
        operation: boolean operation.
        p_s: Perceptron, PerceptronLayer or PerceptronNetwork.

    Raises:
        ValueError: if operation and p_s do not expect same number of inputs."""
    if len(signature(operation).parameters) != p_s.expected_number_of_inputs():
        raise ValueError(f"""Operation amount of arguments do not match perceptron
expected amount of inputs.""")

In [121]:
def perceptron_truth_table(operation: Callable,
                           p_s: ml.InOutPutNetworkI) -> pd.DataFrame:
    """Construct a truth table which contains both the result for a boolean operation, and a Perceptron/PerceptronLayer/PerceptronNetwork,
    for all possible boolean inputs.

    Args:
        operation: boolean operation. Should return results as integers, in tuple if multiple result.
        p_s: Perceptron, PerceptronLayer or PerceptronNetwork.

    Returns:
        DataFrame with inputs, operation result named "Operation Result", p_s result named "Perceptron Result".

    Raises:
        ValueError: if operation and p_s do not expect same number of inputs."""

    verify_perceptron_operation_inputs(operation, p_s)

    input_space = binary_input_space(p_s.expected_number_of_inputs())

    true_result = apply_operation(input_space, operation)
    perceptron_result = apply_perceptron(input_space, p_s)

    input_space["Operation Result"] = true_result
    input_space["Perceptron Result"] = perceptron_result

    return input_space

The first 4 logical gates speak for themselves, and can be tested easily using the framework we established earlier without much explanation.

<h4>1. INVERT gate</h4>

In [122]:
invert_weights = np.array([-1])
invert_bias = 0.5

invert_gate = ml.Perceptron(invert_weights, invert_bias)

In [123]:
invert_table = perceptron_truth_table(lambda a: int(not a),
                                      invert_gate)

invert_table

,Input 0,Operation Result,Perceptron Result
0,0,1,1
1,1,0,0


In [124]:
(invert_table["Operation Result"] == invert_table["Perceptron Result"]).all()

True

The perceptron produces the expected output.

<h4>2. AND gate</h4>

In [125]:
and_weights = np.array([0.5, 0.5])
and_bias = -1

and_gate = ml.Perceptron(and_weights, and_bias)

In [126]:
and_table = perceptron_truth_table(lambda a, b: int(a and b),
                                   and_gate)

and_table

,Input 0,Input 1,Operation Result,Perceptron Result
0,0,0,0,0
1,0,1,0,0
2,1,0,0,0
3,1,1,1,1


In [127]:
(and_table["Operation Result"] == and_table["Perceptron Result"]).all()

True

The perceptron produces the expected output.

<h4>3. OR gate</h4>

In [128]:
or_weights = np.array([1, 1])
or_bias = -1

or_gate = ml.Perceptron(or_weights, or_bias)

In [129]:
or_table = perceptron_truth_table(lambda a, b: int(a or b),
                                   or_gate)

or_table

,Input 0,Input 1,Operation Result,Perceptron Result
0,0,0,0,0
1,0,1,1,1
2,1,0,1,1
3,1,1,1,1


In [130]:
(or_table["Operation Result"] == or_table["Perceptron Result"]).all()

True

The perceptron produces the expected output.

<h4>4. NOR gate</h4>

In [131]:
nor_weights = np.array([-1, -1, -1])
nor_bias = 0

nor_gate = ml.Perceptron(nor_weights, nor_bias)

In [132]:
nor_table = perceptron_truth_table(lambda a, b, c: int((not a) and (not b) and (not c)),
                                   nor_gate)

nor_table

,Input 0,Input 1,Input 2,Operation Result,Perceptron Result
0,0,0,0,1,1
1,0,0,1,0,0
2,0,1,0,0,0
3,0,1,1,0,0
4,1,0,0,0,0
5,1,0,1,0,0
6,1,1,0,0,0
7,1,1,1,0,0


In [133]:
(nor_table["Operation Result"] == nor_table["Perceptron Result"]).all()

True

The perceptron produces the expected output.

<h4>5. A non-specified "more complex" decision system with at least 3 inputs.</h4>

For this, I'll be implementing something I'll call an <i>NOTSTEVE gate</i>. It has 3 binary inputs, representing your 3 cousins: Adam(0), John(1) and Steve(2). You are the type of person that enjoys going to any family gathering, regardless of which of your cousins is there, because the food is wonderful. But right now, you owe Steve some money, and would not like to be at the next family gathering if he's there. Luckily, your cousins Adam and John owe him more money, so you would like to go if they come too, because that would distract Steve enough for you to enjoy your dinner in relative peace. In summary that means: <b>the perceptron should output 0 if input 2 = 1, unless inputs 0 & 1 are also 1. In any other case, it should output 1</b>. For validation purposes, we can summarize this in a boolean operation like before:

In [134]:
def notsteve_operation(adam: bool, john: bool, steve: bool) -> bool:
    # FIXME(m-jeu): Steve could be removed from 2nd boolean expression because of short-circuited or?
    return 1 if (not steve) or (adam and john and steve) else 0


Then we can also instantiate a perceptron with the required weights to be functionally identical to the notsteve-gate:

In [135]:
notsteve_weights = np.array([0.4, 0.4, -1])
notsteve_bias = 0.5

notsteve_gate = ml.Perceptron(notsteve_weights, notsteve_bias)

In [136]:
notsteve_table = perceptron_truth_table(notsteve_operation,
                                        notsteve_gate)

notsteve_table

,Input 0,Input 1,Input 2,Operation Result,Perceptron Result
0,0,0,0,1,1
1,0,0,1,0,0
2,0,1,0,1,1
3,0,1,1,0,0
4,1,0,0,1,1
5,1,0,1,0,0
6,1,1,0,1,1
7,1,1,1,1,1


In [137]:
(notsteve_table["Operation Result"] == notsteve_table["Perceptron Result"]).all()

True

The perceptron produces the expected output.

<h2>Perceptron network</h2>

Secondly, we're required to test the implementation of the implementation of the Perceptron network (and indirectly, the Perceptron layer). The perceptron layer combines one or more perceptrons, to form a layer capable of applying input to all perceptrons contained within it, and creating a new output from this. The perceptron network combines one or more of these layers into a feedforward-network.

The implementation tests consist of implementing:

<ol>
<li>A XOR-gate</li>
<li>A half-adder</li>
</ol>

<h4>XOR Gate</h4>

XOR-gates cannot be implemented with a single Perceptron, but rather with at least 3. We can use 1 layer consisting of a NAND-gate and an OR gate, and then combining these in a second layer with an AND gate.

We already have an AND-gate, and an OR-gate. Therefore, we only need to create a NAND-gate.

In [138]:
nand_weights = np.array([-0.9, -0.9])
nand_bias = 1
nand_gate = ml.Perceptron(nand_weights, nand_bias)
nand_gate

Perceptron: b: 1 )

In [139]:
xor_layer_one = ml.PerceptronLayer(np.array([nand_gate, or_gate]))
xor_layer_two = ml.PerceptronLayer(np.array([and_gate]))
xor_network = ml.PerceptronNetwork(np.array([xor_layer_one, xor_layer_two]))


In [140]:
xor_table = perceptron_truth_table(lambda a, b: int(a ^ b),
                                   xor_network)

xor_table

,Input 0,Input 1,Operation Result,Perceptron Result
0,0,0,0,[0]
1,0,1,1,[1]
2,1,0,1,[1]
3,1,1,0,[0]


In [141]:
(xor_table["Operation Result"] == xor_table["Perceptron Result"]).all()

True

The perceptron network produces the expected output.

<h4>Half-adder</h4>

The half-adder is not a logic gate like the ones before, it produces multiple outputs. It adds 2 1-bit binary numbers, and outputs the result as a 2-bit binary number. We need 2 binary outputs to represent the 2-bit number. By accounting for this in the boolean operation, the framework can handle the rest.

The half-adder works internally with the first digit being an AND operation performed on the inputs, and the second digit being a XOR operation. We implemented these before, but the easiest way to implement it as a dense network, is by specifying the individual perceptrons again:

In [142]:
half_adder_operation = lambda a, b: (int(a and b), int(a ^ b))

In [143]:
ha_layer_1 = ml.PerceptronLayer(np.array([
    ml.Perceptron(np.array([1, 1]), -2),
    ml.Perceptron(np.array([1, 1]), -1),
    ml.Perceptron(np.array([-1, -1]), 1)
    ])
)

ha_layer_2 = ml.PerceptronLayer(np.array([
    ml.Perceptron(np.array([1, 0, 0]), -1),
    ml.Perceptron(np.array([0, 1, 1]), -2)
    ])
)

ha = ml.PerceptronNetwork(np.array([ha_layer_1, ha_layer_2]))

In [144]:
ha_table = perceptron_truth_table(half_adder_operation,
                                  ha)

ha_table

,Input 0,Input 1,Operation Result,Perceptron Result
0,0,0,"(0, 0)","(0, 0)"
1,0,1,"(0, 1)","(0, 1)"
2,1,0,"(0, 1)","(0, 1)"
3,1,1,"(1, 0)","(1, 0)"


In [145]:
(ha_table["Operation Result"] == ha_table["Perceptron Result"]).all()

True

The perceptron network produces the expected output.